In [1]:
import numpy as np
import pandas as pd



In [2]:
resultsDF = pd.read_csv("../../evaluation/results/result_raw", sep=",", header=0)
resultsDF.head()

,alg,group_id,user_id,metric,result
0,mf_30_1.0_avg_sim_group_2_AVG,0_1,3456,AR,2.734497
1,mf_30_1.0_avg_sim_group_2_AVG,0_1,3456,nDCG,0.851243
2,mf_30_1.0_avg_sim_group_2_AVG,0_1,5488,AR,2.735564
3,mf_30_1.0_avg_sim_group_2_AVG,0_1,5488,nDCG,0.784615
4,mf_30_1.0_avg_sim_group_2_AVG,1_1,4201,AR,1.704677


In [3]:
resultsDF.shape

(2970360, 5)

### Show AVG result just to ensure that none of the algorithms went haywire

In [4]:
groupDF = resultsDF.groupby(["alg","group_id","user_id","metric"]).agg("mean")[["result"]]
groupDF.reset_index(inplace=True)
groupDF.head()

,alg,group_id,user_id,metric,result
0,mf_30_1.0_avg_div_group_2_AVG,0_1,3148,AR,1.431856
1,mf_30_1.0_avg_div_group_2_AVG,0_1,3148,nDCG,0.621247
2,mf_30_1.0_avg_div_group_2_AVG,0_1,4075,AR,1.991520
3,mf_30_1.0_avg_div_group_2_AVG,0_1,4075,nDCG,0.709380
4,mf_30_1.0_avg_div_group_2_AVG,0_2,3148,AR,1.281817


In [5]:
groupDF.shape

(2970360, 5)

In [6]:
groupDF.to_csv("../../evaluation/results/res_group_folds.csv", sep=",", header=True)

In [7]:
from pathlib import Path
import sys, os
from typing import Dict, List, NamedTuple, Tuple
import numpy as np
from collections import defaultdict
import time


In [8]:
def compute_metrics(alg_name, groups, results):
    # test_data are triplets: user_id, item_id, and rating
    #LP: test data is matrix user_id x item_id !!!!!! a ja si rikal, jakto ze ti to prirazeni funguje...

    avg_rating = []
    min_rating = []
    minmax_rating = []
    std_rating = []
    
    avg_nDCG_rating = []
    min_nDCG_rating = []
    minmax_nDCG_rating = []
    std_nDCG_rating = []
        
    for gid in groups:
        gid_res_ndcg = results.loc[(results.group_id == gid)&(results.metric == "nDCG")]
        gid_res_AR = results.loc[(results.group_id == gid)&(results.metric == "AR")] 
        
        #print(gid_res_ndcg)
        #print(gid_res_AR)
        
        group_users_mean_ratings = [i for i in gid_res_AR.result]
        group_users_ndcg_ratings = [i for i in gid_res_ndcg.result]
        
        #print(group_users_mean_ratings)
        if len(group_users_mean_ratings) > 0:
            avg_rating.append(np.average(group_users_mean_ratings)) 
            min = np.min(group_users_mean_ratings)
            min_rating.append(min) 
            max = np.max(group_users_mean_ratings)
            minmax_rating.append(0.0 if max == 0.0 else min/max)
            std_rating.append(np.std(group_users_mean_ratings)) 

            avg_nDCG_rating.append(np.average(group_users_ndcg_ratings)) 
            min = np.min(group_users_ndcg_ratings)
            min_nDCG_rating.append(min) 
            max = np.max(group_users_ndcg_ratings)
            minmax_nDCG_rating.append(0.0 if max == 0.0 else min/max)
            std_nDCG_rating.append(np.std(group_users_ndcg_ratings)) 

    print(len(avg_rating))    
    results = pd.DataFrame({
       "AR_avg": avg_rating ,
        "AR_min": min_rating ,
        "AR_min/max": minmax_rating ,
        #"AR_std": std_rating ,
        
        "nDCG_avg": avg_nDCG_rating ,
        "nDCG_min": min_nDCG_rating ,
        "nDCG_min/max": minmax_nDCG_rating #,
        #"nDCG_std": std_nDCG_rating 
    })
      
    return results

In [9]:
algs = resultsDF.alg.unique()
groups = resultsDF.group_id.unique()
metrics = resultsDF.metric.unique()

In [12]:
from scipy import stats
#Comparison for size 8
def calculate_significance(results, grouptype, size, algs):
    print("")
    print(grouptype,size)

    otherAlgs = [a for a in algs if ( grouptype+"_group_"+size in a )]

    for count1, alg1 in enumerate(otherAlgs):
        baseDF = results[alg1]
        
        for count2,alg in enumerate(otherAlgs):
            if count1 != count2:
                otherDF = results[alg]
                res = {}
                for col in otherDF.columns:
                    mean_first = np.mean(baseDF.loc[:,col])
                    mean_second = np.mean(otherDF.loc[:,col])
                    mean_diff = np.mean(baseDF.loc[:,col] - otherDF.loc[:,col])
                    stat,pval = stats.ttest_rel(baseDF.loc[:,col], otherDF.loc[:,col])
                    if pval > 0.0001:
                        print("pval_excess",col)
                    res[col] = (mean_first,mean_second,mean_diff, pval)

                alg1 = alg1.replace("mf_30_1.0_avg_","")
                alg1 = alg1.replace(grouptype+"_group_"+size+"_","")
                alg = alg.replace("mf_30_1.0_avg_","")
                alg = alg.replace(grouptype+"_group_"+size+"_","")
                print(alg1+" - "+ alg)
                print(pd.DataFrame(res, index=["first","second","diff","pval"]))
                print("")

In [13]:
results_all = {}
algs_sign = []
whitelist = ['GFAR', '_AVG', 'FuzzyDHondtDirectOptimize_1', 'GreedyLM',  'FuzzyDHondt_1',  'SPGreedy',  'fai',  'xpo']
for grouptype in ["sim","div"]:
    for size in ["2","3","4","8"]:
        fileName = "../../evaluation/results/resultRaw_"+grouptype+"_"+size
        resultsDF = pd.read_csv(fileName, sep=",", header=None, names=["alg","group_id","user_id","metric","result"])
        algs = resultsDF.alg.unique()        
        metrics = resultsDF.metric.unique()
        groups = resultsDF.group_id.unique()        
        results = {}
        for a in algs:
            for item in whitelist:
                if item in a:
                    algs_sign.append(a)
                    resPerAlg = resultsDF.loc[resultsDF.alg == a]            
                    results[a] = compute_metrics(a, groups, resPerAlg)
        calculate_significance(results, grouptype, size, algs_sign)

4710
4710
4710
4710
4710
4710
4710
4710
4710

sim 2
AVG - FuzzyDHondtDirectOptimize_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg       nDCG_min  nDCG_min/max
first   2.823702  2.428369    0.762617  0.839555   7.932508e-01  8.969116e-01
second  2.724982  2.690628    0.973120  0.825348   7.719553e-01  8.813602e-01
diff    0.098720 -0.262259   -0.210502  0.014207   2.129549e-02  1.555141e-02
pval    0.000000  0.000000    0.000000  0.000000  3.458169e-206  1.020839e-36

AVG - FuzzyDHondt_1.0
              AR_avg        AR_min    AR_min/max      nDCG_avg      nDCG_min  \
first   2.823702e+00  2.428369e+00  7.626175e-01  8.395553e-01  7.932508e-01   
second  2.763327e+00  2.371684e+00  7.580527e-01  8.230150e-01  7.747680e-01   
diff    6.037557e-02  5.668495e-02  4.564820e-03  1.654032e-02  1.848275e-02   
pval    1.383762e-38  1.284709e-36  5.508708e-09  3.731906e-38  3.412298e-38   

        nDCG_min/max  
first   8.969116e-01  
second  8.858523e-01  
diff    1.105936e-02  
pval  

pval    8.174446e-31  

pval_excess nDCG_min/max
FuzzyDHondt_1.0 - mf_30_1.0_fai_sim_group_2
          AR_avg         AR_min     AR_min/max  nDCG_avg  nDCG_min  \
first   2.763327   2.371684e+00   7.580527e-01  0.823015  0.774768   
second  2.424960   2.213186e+00   8.430936e-01  0.715686  0.670685   
diff    0.338366   1.584980e-01  -8.504092e-02  0.107329  0.104083   
pval    0.000000  7.642631e-130  1.667261e-271  0.000000  0.000000   

        nDCG_min/max  
first       0.885852  
second      0.884630  
diff        0.001222  
pval        0.490141  

FuzzyDHondt_1.0 - mf_30_1.0_xpo_sim_group_2
          AR_avg        AR_min     AR_min/max  nDCG_avg  nDCG_min  \
first   2.763327  2.371684e+00   7.580527e-01  0.823015  0.774768   
second  2.449098  2.240688e+00   8.464058e-01  0.693213  0.643239   
diff    0.314229  1.309960e-01  -8.835314e-02  0.129802  0.131529   
pval    0.000000  3.182926e-93  1.342057e-307  0.000000  0.000000   

        nDCG_min/max  
first   8.858523e-01  
seco

pref_AVG - SPGreedy_1.0
          AR_avg    AR_min     AR_min/max      nDCG_avg  nDCG_min  \
first   2.705296  2.122270   6.657089e-01  8.066943e-01  0.678858   
second  2.810955  2.450165   7.793072e-01  8.139777e-01  0.771581   
diff   -0.105659 -0.327896  -1.135983e-01 -7.283416e-03 -0.092723   
pval    0.000000  0.000000  5.754247e-282  6.635517e-26  0.000000   

        nDCG_min/max  
first       0.733789  
second      0.901703  
diff       -0.167914  
pval        0.000000  

pref_AVG - mf_30_1.0_fai_sim_group_2
          AR_avg        AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   2.705296  2.122270e+00    0.665709  0.806694  0.678858      0.733789
second  2.424960  2.213186e+00    0.843094  0.715686  0.670685      0.884630
diff    0.280336 -9.091623e-02   -0.177385  0.091008  0.008173     -0.150841
pval    0.000000  5.239234e-36    0.000000  0.000000  0.000094      0.000000

pref_AVG - mf_30_1.0_xpo_sim_group_2
          AR_avg        AR_min  AR_min/max  nDCG_avg  

mf_30_1.0_xpo_sim_group_2 - pref_AVG
          AR_avg        AR_min  AR_min/max  nDCG_avg      nDCG_min  \
first   2.449098  2.240688e+00    0.846406  0.693213  6.432388e-01   
second  2.705296  2.122270e+00    0.665709  0.806694  6.788577e-01   
diff   -0.256199  1.184183e-01    0.180697 -0.113481 -3.561893e-02   
pval    0.000000  4.513656e-60    0.000000  0.000000  2.614617e-59   

        nDCG_min/max  
first       0.867896  
second      0.733789  
diff        0.134106  
pval        0.000000  

mf_30_1.0_xpo_sim_group_2 - SPGreedy_1.0
          AR_avg    AR_min     AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   2.449098  2.240688   8.464058e-01  0.693213  0.643239  8.678955e-01
second  2.810955  2.450165   7.793072e-01  0.813978  0.771581  9.017033e-01
diff   -0.361858 -0.209477   6.709863e-02 -0.120765 -0.128342 -3.380775e-02
pval    0.000000  0.000000  1.750711e-229  0.000000  0.000000  2.505959e-90

mf_30_1.0_xpo_sim_group_2 - mf_30_1.0_fai_sim_group_2
               AR_av

GFAR_1.0 - AVG
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   2.125283  1.839571    0.764616  0.651824  0.575529  7.996202e-01
second  2.562275  2.025987    0.659948  0.751832  0.669852  8.105662e-01
diff   -0.436992 -0.186416    0.104668 -0.100008 -0.094324 -1.094598e-02
pval    0.000000  0.000000    0.000000  0.000000  0.000000  1.356649e-08

GFAR_1.0 - FuzzyDHondtDirectOptimize_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min   nDCG_min/max
first   2.125283  1.839571    0.764616  0.651824  0.575529   7.996202e-01
second  2.460295  2.374246    0.931647  0.738026  0.673890   8.430249e-01
diff   -0.335012 -0.534675   -0.167031 -0.086201 -0.098361  -4.340473e-02
pval    0.000000  0.000000    0.000000  0.000000  0.000000  1.365929e-122

GFAR_1.0 - FuzzyDHondt_1.0
          AR_avg        AR_min  AR_min/max       nDCG_avg       nDCG_min  \
first   2.125283  1.839571e+00    0.764616   6.518242e-01   5.755285e-01   
second  2.451978  1.929132e+

...

SPGreedy_1.0 - AVG
          AR_avg         AR_min  AR_min/max  nDCG_avg      nDCG_min  \
first   1.842697   9.603217e-01    0.360593  0.559507  4.029393e-01   
second  1.875972   9.215946e-01    0.334611  0.586698  3.913481e-01   
diff   -0.033275   3.872713e-02    0.025982 -0.027191  1.159116e-02   
pval    0.000000  3.406865e-199    0.000000  0.000000  3.006494e-66   

        nDCG_min/max  
first       0.582967  
second      0.524673  
diff        0.058294  
pval        0.000000  

SPGreedy_1.0 - FuzzyDHondtDirectOptimize_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   1.842697  0.960322    0.360593  0.559507  0.402939      0.582967
second  1.688798  1.429653    0.760152  0.575250  0.476655      0.707555
diff    0.153899 -0.469331   -0.399559 -0.015743 -0.073716     -0.124587
pval    0.000000  0.000000    0.000000  0.000000  0.000000      0.000000

pval_excess nDCG_avg
SPGreedy_1.0 - FuzzyDHondt_1.0
              AR_avg         AR_min     AR_min

5000
5000
5000
5000
5000
5000
5000
5000
5000

div 8
AVG - FuzzyDHondtDirectOptimize_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first   1.622016  0.655491    0.249810  0.499888  0.276275      0.404083
second  1.490029  0.953696    0.505486  0.487921  0.360630      0.584903
diff    0.131987 -0.298206   -0.255676  0.011967 -0.084355     -0.180819
pval    0.000000  0.000000    0.000000  0.000000  0.000000      0.000000

AVG - FuzzyDHondt_1.0
               AR_avg         AR_min     AR_min/max       nDCG_avg  \
first    1.622016e+00   6.554907e-01   2.498102e-01   4.998879e-01   
second   1.457028e+00   5.645363e-01   2.219122e-01   4.517463e-01   
diff     1.649877e-01   9.095434e-02   2.789800e-02   4.814160e-02   
pval    3.335286e-190  1.715747e-152  1.361652e-112  1.837167e-188   

             nDCG_min   nDCG_min/max  
first    2.762749e-01   4.040834e-01  
second   2.381322e-01   3.548296e-01  
diff     3.814269e-02   4.925380e-02  
pval    2.033434e

pval    6.366431e-112  

FuzzyDHondt_1.0 - SPGreedy_1.0
              AR_avg         AR_min  AR_min/max      nDCG_avg       nDCG_min  \
first   1.457028e+00   5.645363e-01    0.221912  4.517463e-01   2.381322e-01   
second  1.533382e+00   6.928109e-01    0.288435  4.612207e-01   2.813752e-01   
diff   -7.635379e-02  -1.282745e-01   -0.066523 -9.474490e-03  -4.324299e-02   
pval    2.583104e-44  6.896704e-250    0.000000  3.467726e-09  5.170498e-172   

        nDCG_min/max  
first       0.354830  
second      0.455303  
diff       -0.100473  
pval        0.000000  

pval_excess nDCG_min
FuzzyDHondt_1.0 - mf_30_1.0_fai_div_group_8
               AR_avg         AR_min  AR_min/max  nDCG_avg  nDCG_min  \
first    1.457028e+00   5.645363e-01    0.221912  0.451746  0.238132   
second   1.251730e+00   6.602455e-01    0.354760  0.384045  0.237753   
diff     2.052982e-01  -9.570912e-02   -0.132848  0.067701  0.000380   
pval    3.223643e-264  2.587457e-116    0.000000  0.000000  0.826134   

 

pref_AVG - GFAR_1.0
          AR_avg    AR_min  AR_min/max  nDCG_avg       nDCG_min  nDCG_min/max
first   1.526135  0.556613    0.205329  0.470645   2.167213e-01      0.297024
second  1.387317  0.698432    0.340117  0.448862   2.482973e-01      0.396159
diff    0.138819 -0.141819   -0.134789  0.021784  -3.157604e-02     -0.099135
pval    0.000000  0.000000    0.000000  0.000000  5.084348e-130      0.000000

pref_AVG - GreedyLM_1.0
               AR_avg    AR_min  AR_min/max  nDCG_avg  nDCG_min  nDCG_min/max
first    1.526135e+00  0.556613    0.205329  0.470645  0.216721      0.297024
second   1.554759e+00  0.752781    0.313055  0.486584  0.318459      0.501523
diff    -2.862329e-02 -0.196168   -0.107726 -0.015939 -0.101737     -0.204498
pval    4.097995e-150  0.000000    0.000000  0.000000  0.000000      0.000000

pref_AVG - SPGreedy_1.0
              AR_avg    AR_min  AR_min/max       nDCG_avg  nDCG_min  \
first   1.526135e+00  0.556613    0.205329   4.706455e-01  0.216721   
second  

mf_30_1.0_xpo_div_group_8 - FuzzyDHondt_1.0
              AR_avg         AR_min  AR_min/max      nDCG_avg  nDCG_min  \
first   1.396267e+00   6.893907e-01    0.331679  4.415452e-01  0.244959   
second  1.457028e+00   5.645363e-01    0.221912  4.517463e-01  0.238132   
diff   -6.076152e-02   1.248544e-01    0.109767 -1.020101e-02  0.006827   
pval    3.547703e-28  5.688783e-188    0.000000  3.008303e-10  0.000053   

        nDCG_min/max  
first   3.985993e-01  
second  3.548296e-01  
diff    4.376969e-02  
pval    1.416425e-61  

pval_excess nDCG_min/max
mf_30_1.0_xpo_div_group_8 - GFAR_1.0
              AR_avg        AR_min    AR_min/max       nDCG_avg  nDCG_min  \
first   1.396267e+00  6.893907e-01  3.316793e-01   4.415452e-01  0.244959   
second  1.387317e+00  6.984324e-01  3.401175e-01   4.488620e-01  0.248297   
diff    8.950137e-03 -9.041746e-03 -8.438182e-03  -7.316702e-03 -0.003338   
pval    3.067675e-40  1.502787e-07  1.069505e-16  4.721259e-150  0.000002   

        nDCG_min